In [2]:
# Importing libraries and establishing connection to SQL database

import numpy as np
import pandas as pd
import sqlite3 as sql
import difflib
data_path = '../zippedData'
conn = sql.connect(data_path + '/im.db')

In [3]:
# Declaration of base dataframes

# Studios and gross revenue - 3387 rows
# https://www.boxofficemojo.com/
gross_df = pd.read_csv(data_path + '/bom.movie_gross.csv')

# The movie database - https://www.themoviedb.org/?language=en-US
# Reviews - 26517 rows, release date, popularity score, votes?
reviews_df = pd.read_csv(data_path + '/tmdb.movies.csv')

# Budget and gross revenues - 5782 rows
# https://www.the-numbers.com/
budget_df = pd.read_csv(data_path + '/tn.movie_budgets.csv')

# Ratings and genres by director from imdb database
director_and_ratings_df = pd.read_sql(
"""SELECT DISTINCT b.*,
            r.averagerating AS avg_rating,
            r.numvotes AS num_votes,
            p.primary_name AS director,
            p.primary_profession
FROM movie_basics b
JOIN movie_ratings r
ON b.movie_id = r.movie_id
LEFT JOIN directors d
ON b.movie_id = d.movie_id
LEFT JOIN persons p
ON d.person_id = p.person_id
""", conn)

# Ratings and genres by writers from imdb database
writers_df = pd.read_sql("""
SELECT DISTINCT b.*, r.averagerating AS avg_rating, r.numvotes AS num_votes, p.primary_name AS 'writer', p.primary_profession
FROM movie_basics b
JOIN movie_ratings r
ON b.movie_id = r.movie_id
LEFT JOIN writers w
ON b.movie_id = w.movie_id
LEFT JOIN persons p
ON w.person_id = p.person_id

""", conn)

In [4]:
# Reformatting data in budget_df

# Turned all the monetary strings into ints by replacing $ and . with empty strings
budget_df[budget_df.columns[3:]] = budget_df[budget_df.columns[3:]].replace('[\$\.]|,', '', regex=True).astype('int')

# Added column of world wide gross - production budget to get the net profit
budget_df['profit'] = budget_df['worldwide_gross'] - budget_df['production_budget']

In [5]:
# Merging budget dataframe and gross dataframe by domestic gross
# Merged on movie from budget_df and title from gross_df
budg_and_gross_df = pd.merge(budget_df, gross_df, how = "outer", left_on = 'movie', right_on = 'title')

# Dropping id and year columns, since year is represented in release_date
budg_and_gross_df.drop(['id', 'year'], axis=1, inplace=True)

# Merging gross_y and gross_x columns, preferring gross_x which seem to have higher precision
budg_and_gross_df.domestic_gross_x = budg_and_gross_df.domestic_gross_x.fillna(budg_and_gross_df.domestic_gross_y)

# Merging movie and title, preferring movie
budg_and_gross_df.movie = budg_and_gross_df.movie.fillna(budg_and_gross_df.title)

# Dropping redundant gross and title columns, dropping unnecessary studio and foreign gross columns
budg_and_gross_df.drop(['domestic_gross_y', 'title', 'studio', 'foreign_gross'], axis=1, inplace=True)

# Change dates to formatted strings
budg_and_gross_df['release_date'] = pd.to_datetime(budg_and_gross_df['release_date']).dt.strftime('%Y-%m-%d')

# New column for year
budg_and_gross_df['year'] = budg_and_gross_df['release_date'].str.split('-').str[0]

# Relabeling columns
budg_and_gross_df.columns = ['release_date', 'title', 'prod_budget', 'dom_gross', 'world_gross', 'profit', 'year']

# Reformatting money columns to integers
money_columns = ['prod_budget', 'dom_gross', 'world_gross', 'profit']
budg_and_gross_df[money_columns] = budg_and_gross_df[money_columns].astype('Int64')

# Keep only entries in budg_and_gross where prod_budget is not null
budg_and_gross_df = budg_and_gross_df.loc[budg_and_gross_df.prod_budget.notna()]

In [6]:
# Merging writers and directors dataframes on movie_id
person_df = pd.merge(writers_df, director_and_ratings_df, how='inner', on='movie_id')

In [7]:
# We want to merge these 3 tables onto each other using titles
# Standardizing the title columns of each dataframe to maximize the number of overlapping title keys
# First, replace all non alphanumeric characters with spaces, then turn any number of spaces into a single space
# Finally, strip any leading or trailing white spaces and convert all characters to lowercase
budg_and_gross_df.title = budg_and_gross_df.title.str.replace('\W', ' ', regex=True).str.replace('\s+', ' ', regex=True).str.strip().str.lower()
person_df.primary_title_x = person_df.primary_title_x.str.replace('\W', ' ', regex=True).str.replace('\s+', ' ', regex=True).str.strip().str.lower()
reviews_df.title = reviews_df.title.str.replace('\W', ' ', regex=True).str.replace('\s+', ' ', regex=True).str.strip().str.lower()

In [8]:
# Convert start year from person_df into string to match budg_and_gross_df
person_df.start_year_x = person_df.start_year_x.astype(str)

In [9]:
# Merge budg_and_gross_df and reviews_df, on titles then on release date
# to prevent movies with the same name but different dates from overwriting
# Merge left to preserve data from budg_and_gross which may not appear in reviews_df
merge_reviews = pd.merge(budg_and_gross_df, reviews_df, how='left', left_on=['title', 'release_date'] , right_on=['title', 'release_date'])

# Merge merge_reviews and person_df, on titles then on release date
# to prevent movies with the same name but different dates from overwriting
# Merge left to preserve data from merge_reviews which may not appear in person_df
merge_reviews_directors = pd.merge(merge_reviews, person_df, how='left', left_on=['title', 'year'], right_on=['primary_title_x','start_year_x'])

In [10]:
# Selecting columns to drop in order to clean up data frame
columns_to_keep = ['release_date','prod_budget','title', 'dom_gross', 'world_gross', 'profit', 'genre_ids', 'runtime_minutes_x', 'genres_x', 'writer', 'director']
columns_to_drop = [x for x in list(merge_reviews_directors.columns) if x not in columns_to_keep]

# Dropping columns based on above selection
merge_reviews_directors.drop(columns_to_drop, axis=1, inplace=True)

In [11]:
# Renaming columns
column_names = ['release_date', 'title', 'prod_budget', 'dom_gross', 'world_gross', 'profit', 'genre_ids', 'runtime_min', 'genres', 'writer', 'director']
merge_reviews_directors.columns = column_names

# Reorganizing columns
merge_reviews_directors = merge_reviews_directors.reindex(columns=['title', 'profit', 'prod_budget', 'dom_gross', 'world_gross', 'genre_ids', 'genres','director','writer','runtime_min','release_date'])

# Dropping duplicate rows
merge_reviews_directors.drop_duplicates(inplace=True)

In [12]:
# From https://www.themoviedb.org/talk/5daf6eb0ae36680011d7e6ee
# Construct list of dictionaries to decipher genre_ids column
genre_dictionary = [{"id":28,"name":"Action"},
                              {"id":12,"name":"Adventure"},
                              {"id":16,"name":"Animation"},
                              {"id":35,"name":"Comedy"},
                              {"id":80,"name":"Crime"},
                              {"id":99,"name":"Documentary"},
                              {"id":18,"name":"Drama"},
                              {"id":10751,"name":"Family"},
                              {"id":14,"name":"Fantasy"},
                              {"id":36,"name":"History"},
                              {"id":27,"name":"Horror"},
                              {"id":10402,"name":"Music"},
                              {"id":9648,"name":"Mystery"},
                              {"id":10749,"name":"Romance"},
                              {"id":878,"name":"Science Fiction"},
                              {"id":10770,"name":"TV Movie"},
                              {"id":53,"name":"Thriller"},
                              {"id":10752,"name":"War"},
                              {"id":37,"name":"Western"}]

In [13]:
# Decipher genre_ids column

# Currently, each item in column is a string, conver to list as follows:
# Strip the brackets from either end, then replace spaces with empty strings
# Then strip leading and trailing white spaces and split into list along commas
merge_reviews_directors.genre_ids = merge_reviews_directors.genre_ids.str.strip('\[\]').str.replace(" ", "").str.strip().str.split(',')

# Explode dataframe along column ids, creating duplicated rows that differ only by genre_ids value
merge_reviews_directors = merge_reviews_directors.explode('genre_ids')

# Replace empty strings or spaces with np.nan
merge_reviews_directors.genre_ids = merge_reviews_directors.genre_ids.replace(r'^\s*$', np.nan, regex=True)

# Convert entries to floats
merge_reviews_directors.genre_ids.astype('float')

# Define a function that returns 'None' if key is np.nan, otherwise, find id in dictionary and return value
def find_genre(key):
    if key is np.nan:
        return 'None'
    for dictionary in genre_dictionary:
        if dictionary['id'] == int(key):
            return dictionary['name']
    return 'None'

# Apply find_genre function to genre_ids
merge_reviews_directors.genre_ids = merge_reviews_directors.genre_ids.apply(lambda x: find_genre(x))

In [14]:
# Data was exploded due to previous conversion, now we need to collapse it back into singular titles.
# Group by movie title, then aggregate as follows:
# All numerical values by first, date by first
# Condense genre_ids, directors, and writers into lists that can be exploded later (removing duplicates)
# Reset index to return from grouby object into dataframe
merge_reviews_directors = merge_reviews_directors.groupby('title').agg({'profit': 'first',
                                              'prod_budget': 'first',
                                              'dom_gross': 'first',
                                              'world_gross': 'first',
                                              'genre_ids': lambda x: list(set(x.to_list())),
                                              'genres': 'first',
                                              'director': lambda x: list(set(x.to_list())),
                                              'writer': lambda x: list(set(x.to_list())),
                                              'runtime_min': 'first',
                                              'release_date': 'first'}).reset_index()

# Remap NaNs in genres to 'None', as consistent with genre_id row
merge_reviews_directors.genres = merge_reviews_directors.genres.fillna('None')

# Convert genres into a list rather than string
merge_reviews_directors['genres'] = merge_reviews_directors['genres'].str.split(',')

# Have duplicate genres between two lists: Sci-Fi/Science Fiction and Music/Musical
# Remove duplicates from both lists, keeping only Sci-Fi and Musical
# Define helper function
def replace_keys(id_list):
    for index, item in enumerate(id_list):
        if 'Music' == item:
            id_list[index] = 'Musical'
        elif 'Science Fiction' == item:
            id_list[index] = 'Sci-Fi'
    return id_list

merge_reviews_directors.genre_ids = merge_reviews_directors.genre_ids.apply(lambda x: replace_keys(x))
merge_reviews_directors.genres = merge_reviews_directors.genres.apply(lambda x: replace_keys(x))

# Overwrite genres by merging genre_ids and genres columns
merge_reviews_directors.genres = merge_reviews_directors.genre_ids + merge_reviews_directors.genres

# Convert to set and back to list to remove duplicates
merge_reviews_directors.genres = merge_reviews_directors.genres.apply(lambda x: list(set(x)))

# Drop the now redundant genre_ids column
merge_reviews_directors.drop('genre_ids', axis=1, inplace=True)

# Insert ROI column as second column, as profit divided by production budget * 100 (as percent, not decimal)
merge_reviews_directors.insert(loc=1, column='ROI', value = ((merge_reviews_directors.profit / merge_reviews_directors.prod_budget) * 100))

In [15]:
# Write to merged_data.csv for further use in other notebooks
# merge_reviews_directors.to_csv('../zippedData/merged_data.csv', index=False)

In [16]:
# Converting release_date column to datetime objects
merge_reviews_directors.release_date = pd.to_datetime(merge_reviews_directors.release_date)

# Locating the top 2 rows by ROI with titles released in the last 10 years (EDA showed that these were outliers)
merge_reviews_directors.loc[merge_reviews_directors.release_date.dt.year >= 2012].nlargest(2,'ROI')

# Dropping 'the gallows' and 'the devil inside' as outliers for 2012 data onwards
# Preserving all information from years before 2012 in case they remain useful
merge_reviews_directors.drop([4467,4366], axis=0, inplace=True)

# Want to remove extra 'None' values that appear in genres when genres already has other values
# Define helper function
def remove_nones_nonempty(genre_list):
    if genre_list == ['None']:
        return genre_list
    elif 'None' in genre_list:
        genre_list.remove('None')
    return genre_list

merge_reviews_directors.genres = merge_reviews_directors.genres.apply(lambda x: remove_nones_nonempty(x))

In [17]:
merge_reviews_directors

,title,ROI,profit,prod_budget,dom_gross,world_gross,genres,director,writer,runtime_min,release_date
0,10 000 b c,156.253027,164065678,105000000,94784201,269065678,[None],[nan],[nan],NaN,2008-03-07
1,10 cloverfield lane,2065.728440,103286422,5000000,72082999,108286422,"[Sci-Fi, Mystery, Thriller, Drama, Horror]",[Dan Trachtenberg],"[Matthew Stuecken, Damien Chazelle, Josh Campb...",103.0,2016-03-11
2,10 days in a madhouse,-99.878200,-11985384,12000000,14616,14616,[Drama],[Timothy Hines],[Timothy Hines],111.0,2015-11-11
3,10 things i hate about you,364.722692,47413950,13000000,38177966,60413950,[None],[nan],[nan],NaN,1999-03-31
4,102 dalmatians,-21.245225,-18058441,85000000,66941559,66941559,[None],[nan],[nan],NaN,2000-11-22
...,...,...,...,...,...,...,...,...,...,...,...
5693,zootopia,579.619744,869429616,150000000,341268248,1019429616,"[Family, Animation, Comedy, Adventure]","[Rich Moore, Jared Bush, Byron Howard]","[Rich Moore, Jennifer Lee, Phil Johnston, Jim ...",108.0,2016-03-04
5694,zulu,-88.473575,-14155772,16000000,0,1844228,"[Thriller, Drama, Crime]",[Jérôme Salle],"[Julien Rappeneau, Caryl Ferey, Jérôme Salle]",110.0,2013-12-31
5695,zwartboek,23.810700,5238354,22000000,4398532,27238354,[None],[nan],[nan],NaN,2007-04-06
5696,ã l intã rieur,-70.135600,-2104068,3000000,0,895932,[None],[nan],[nan],NaN,2008-04-15


In [17]:
# Write to merged_data.parquet for further use in other notebooks (preserves object types in columns)
merge_reviews_directors.to_parquet('../zippedData/merged_data.parquet')